In [177]:
import pandas as pd
import numpy as np
import json, re
from uuid import uuid4

## Torch Modules
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [182]:
## PyTorch Transformer
from transformers import RobertaForSequenceClassification, RobertaConfig, AdamW

In [183]:
df = pd.read_csv('../data/myPersonality/mypersonality_final.csv', encoding='latin1')
print("The size of data is {0}".format(df.shape[0]))
labels = ['cEXT','cNEU','cAGR','cCON','cOPN']

The size of data is 9917


In [184]:
df2 = pd.read_csv('../data/Essays/essays2007.csv', encoding='latin1')
print("The size of data is {0}".format(df2.shape[0]))

The size of data is 2468


In [185]:
df4 = df[['STATUS',labels[0]]]
df4 = df4.rename(columns={'STATUS': "text", labels[0]: "label"})
df4.label = pd.Categorical(pd.factorize(df4.label)[0])
df4.label = df4.label.apply(int)

In [186]:
df3 = df2[['text',labels[0]]]
df3 = df3.rename(columns={labels[0]: "label"})

In [187]:
data = df4.append(df3, ignore_index=True)

In [188]:
data.shape

(12385, 2)

In [189]:
train_size = 0.8
train_dataset=data.sample(frac=train_size,random_state=200).reset_index(drop=True)
test_dataset=data.drop(train_dataset.index).reset_index(drop=True)

In [190]:
print("FULL Dataset: {}".format(data.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

FULL Dataset: (12385, 2)
TRAIN Dataset: (9908, 2)
TEST Dataset: (2477, 2)


In [191]:
config = RobertaConfig.from_pretrained('roberta-base')

In [192]:
model = RobertaForSequenceClassification(config)

In [193]:
from tokenizers import BertWordPieceTokenizer

In [194]:
tokenizer = BertWordPieceTokenizer("../data/bert-large-uncased-vocab.txt", lowercase=True)

In [207]:
def prepare_features(seq_1, max_seq_length = 300, 
             zero_pad = False, include_CLS_token = True, include_SEP_token = True):
    ## Tokenzine Input
    tokens_a = tokenizer.encode(seq_1)

    input_ids = tokens_a.ids
    
    ## Input Mask 
    input_mask = [1] * len(input_ids)
    
    return torch.tensor(input_ids).unsqueeze(0), input_mask

In [208]:
prepare_features("hello, comment ça va?")

(tensor([[  101,  7592,  1010,  7615,  6187, 12436,  1029,   102]]),
 [1, 1, 1, 1, 1, 1, 1, 1])

In [209]:
class Data(Dataset):
    def __init__(self, dataframe):
        self.len = len(dataframe)
        self.data = dataframe
        
    def __getitem__(self, index):
        text = self.data.text[index]
        label = self.data.label[index]
        X, _  = prepare_features(text)
        y = self.data.label[index]
        return X, y
    
    def __len__(self):
        return self.len

In [210]:
training_set = Data(train_dataset)
testing_set = Data(test_dataset)

In [211]:
training_set.__getitem__(0)[0].shape

torch.Size([1, 835])

In [212]:
model(training_set.__getitem__(0)[0])

RuntimeError: index out of range: Tried to access index 514 out of table with 513 rows. at /opt/conda/conda-bld/pytorch_1579022060824/work/aten/src/TH/generic/THTensorEvenMoreMath.cpp:418